In [1]:
# from langchain.llms.openai import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.memory import ConversationSummaryBufferMemory
from langchain.schema.runnable import RunnablePassthrough
 
chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

memory = ConversationSummaryBufferMemory(
    llm=chat,
    max_token_limit=120,
    return_messages=True,
)

movie_summarize_example = [
    {
        "question": "Top Gun",
        "answer": "🛩️👨‍✈️🔥",
    },
    {
        "question": "The Godfather",
        "answer": "👨‍👨‍👦🔫🍝",
    },
    {
        "question": "Soul",
        "answer": "🎷👻💡"
    }
]

movie_summarize_example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{question}"),
        ("ai", "{answer}"),
    ]
)

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=movie_summarize_example_prompt,
    examples=movie_summarize_example,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a movie summarizer. If human gives you movie title, Get the short synopsis, and summarize in to 3 emojis. If you dont know the movie title, answer with person shrugging emjois 3 times. If human ask general question, answer normally. Remember, when you reply with emoji, it always should be 3."),
        example_prompt,
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

def load_memory(_):
    return memory.load_memory_variables({}).get("history", [])

chain = RunnablePassthrough.assign(history=load_memory) | final_prompt | chat

def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )


In [2]:
invoke_chain("Top Gun")


🛩️👨‍✈️🔥

In [3]:
invoke_chain("Avatar")

🌿🌌💙

In [4]:
invoke_chain("blabla  pokemon")

🤷‍♂️🤷‍♂️🤷‍♂️

In [5]:
invoke_chain("lala land")

🎶💃🌟

In [6]:
invoke_chain("What was previous movie name I asked for?")

Top GunThe human mentions "Top Gun" and the AI responds with airplane and pilot emojis, indicating excitement or approval.

In [7]:
invoke_chain("How many movis I asked so far?")

You have asked for 5 movies so far.The human mentions "Top Gun" and the AI responds with airplane and pilot emojis, indicating excitement or approval. When the human brings up "Avatar," the AI responds with emojis representing nature, space, and the color blue.